In [1]:
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('dispCDF40_ver4.2.in.txt', delim_whitespace=True, comment='#')

df = df.rename(columns={
    'length': 'L',
    'width': 'W',
    'long': 'lon'
})

df['L'] = 1000*df['L']
df['W'] = 1000*df['W']
df['depth'] *=1000
df['reference'] = 'center'
df.head()

In [ ]:
df = pd.read_csv('usgs.txt', delim_whitespace=True)
df = df.rename(columns={
    'Lat.': 'lat',
    'Lon.': 'lon'
})
df['L'] = 25000
df['W'] = 16600
df['slip'] *= 0.01
df['depth'] *= 1000
df['reference'] = 'center'
print(df['slip'].min(),df['slip'].max())
print(df['depth'].min(),df['depth'].max())

df.head()

In [ ]:
df.to_csv('earthquake.csv',index=False)

# Easywave

In [ ]:
df_easywave = df.copy()
df_easywave['reference']='C'
df_easywave['depth'] /=1000
df_easywave['L'] /=1000
df_easywave['W'] /=1000

In [ ]:
df_easywave.head()

In [ ]:
f = open('fault.flt','w')

for index, row in df_easywave.iterrows():
    
    rowString = '-location {lon} {lat} {depth} -refpos {reference} -strike {strike} -dip {dip} -rake {rake} -slip {slip} -size {L} {W}\n'
    rowString = rowString.format(**row)
    f.write(rowString)

f.close()

# Geoclaw

In [ ]:
from clawpack.geoclaw import dtopotools

In [ ]:
df_geoclaw = df.copy()

In [ ]:
fault = dtopotools.Fault()
fault.subfaults = []
for index, fault_row in df_geoclaw.iterrows():
    subfault = dtopotools.SubFault()
    subfault.strike = fault_row["strike"]
    subfault.length = fault_row["L"]
    subfault.width = fault_row["W"]
    subfault.depth = fault_row["depth"]
    subfault.slip = fault_row["slip"]
    subfault.rake = fault_row["rake"]
    subfault.dip = fault_row["dip"]
    subfault.longitude = fault_row["lon"]
    subfault.latitude = fault_row["lat"]
    subfault.coordinate_specification = "centroid"
    fault.subfaults.append(subfault)

In [ ]:
df_geoclaw.head()

In [ ]:
slack = 3
lat_min = df["lat"].min() - slack
lat_max = df["lat"].max() + slack
lon_min = df["lon"].min() - slack
lon_max = df["lon"].max() + slack
print(lat_min, lat_max, lon_min, lon_max)

In [ ]:
x = np.linspace(lon_min, lon_max, 100)
y = np.linspace(lat_min, lat_max, 100)
times = [1.]

In [ ]:
fault.create_dtopography(x,y,times)
dtopo = fault.dtopo
dtopo_fname = "earthquake.tt3"
dtopo.write(dtopo_fname, dtopo_type=3)

In [ ]:
dtopo_read = dtopotools.DTopography()
dtopo.read(dtopo_fname, dtopo_type=3)

In [ ]:
x = dtopo.x
y = dtopo.y

In [ ]:
plt.figure(figsize=(12,7))
ax1 = plt.subplot(121)
ax2 = plt.subplot(122)
fault.plot_subfaults(axes=ax1,slip_color=True)
ax1.set_xlim(x.min(),x.max())
ax1.set_ylim(y.min(),y.max())
dtopo.plot_dZ_colors(t=1.,axes=ax2)